In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

from pathlib import Path

In [2]:
path = Path('../../../data/soc-wiki-elec')

df = pd.read_csv(path.joinpath('soc-wiki-elec.edges'), sep=' ', header=None, skiprows=1)

# name columns
df.columns = ['source', 'target', 'sign', 'date']

df.head()

,source,target,sign,date
0,3,30,1,1095171960
1,25,30,-1,1095173580
2,4,30,1,1095174480
3,5,30,1,1095176220
4,6,30,1,1095182880


In [5]:
sum(df.sign == 1) / len(df)

0.7840871944784302

In [6]:
# create network and calculate triadic census
G = nx.from_pandas_edgelist(df, 'source', 'target', 'sign', create_using=nx.DiGraph())

In [7]:
G.is_directed()

True

In [8]:
tcen = nx.triadic_census(G)
tcen

{'003': 59378321389,
 '012': 669997442,
 '102': 19697614,
 '021D': 5796353,
 '021U': 3232045,
 '021C': 2746482,
 '111D': 357440,
 '111U': 558513,
 '030T': 462679,
 '030C': 6795,
 '201': 28288,
 '120D': 45556,
 '120U': 58259,
 '120C': 17667,
 '210': 15275,
 '300': 2119}

In [4]:
df_tcen_path = Path(path.joinpath('triad_census.h5'))

try:
    df_tcen = pd.DataFrame.from_dict(tcen, orient='index',
                        columns=['wiki-elec']).transpose()
except:
    df_tcen_path = Path(path.joinpath('triad_census.h5'))

    df_tcen = pd.read_hdf(df_tcen_path)

df_tcen

,003,012,102,021D,021U,021C,111D,111U,030T,030C,201,120D,120U,120C,210,300
wiki-elec,59378321389,669997442,19697614,5796353,3232045,2746482,357440,558513,462679,6795,28288,45556,58259,17667,15275,2119


In [29]:
# save to hdf
df_tcen.to_hdf(df_tcen_path, key='wiki-elec')

df_tcen.to_hdf(df_tcen_path.name, key='wiki-elec')

/home/piotrfens/Desktop/programming/data-analysis/.datavenv/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'wiki-elec'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [3]:
df_tcen2 = pd.read_hdf(df_tcen_path)
df_tcen2

NameError: name 'df_tcen_path' is not defined

In [5]:
df_tcen_path.name

'triad_census.h5'

In [16]:
sum(sum(df_tcen.values))
sum(df_tcen.values)
real_triads = df_tcen[['030T', '030C', '120D', '120U', '120C', '210', '300']]
transitive_triads = df_tcen[['030T', '120D', '120U', '120C', '210', '300']]
sum(sum(real_triads.values)), sum(sum(transitive_triads.values))

(np.int64(608350), np.int64(601555))